##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimator から Keras API に移行する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/migrating_estimator">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/migrating_estimator.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

このガイドでは、TensorFlow 1 の `tf.estimator.Estimator` API から TensorFlow 2 の `tf.keras` API に移行する方法を示します。最初に、`tf.estimator.Estimator` を使用してトレーニングと評価のための基本モデルをセットアップして実行します。次に、`tf.keras` API を使用して  TensorFlow 2 で同等の手順を実行します。また、`tf.GradientTape` をサブクラス化し、`tf.keras.Model` を使用してトレーニングの手順をカスタマイズする方法も学びます。

- TensorFlow 1 では、高レベルの `tf.estimator.Estimator` API を使用して、モデルのトレーニングと評価、推論の実行、およびモデルの保存（提供用）を行うことができます。
- TensorFlow 2 では、Keras API を使用して、[モデルの構築](https://www.tensorflow.org/guide/keras/custom_layers_and_models)、勾配の適用、 [トレーニング](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)、評価、予測などの前述のタスクを実行します。

（モデル/チェックポイント保存ワークフローを TensorFlow 2 に移行するには、[SavedModel](saved_model.ipynb) および [Checkpoint](checkpoint_saved.ipynb) 移行ガイドを確認してください。）

## セットアップ

インポートと単純なデータセットから始めます。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator.Estimator でトレーニングと評価を行う

この例では、TensorFlow 1 で `tf.estimator.Estimator` を使用してトレーニングと評価を実行する方法を示します。

いくつかの関数を定義することから始めます。トレーニングデータの入力関数、評価データの評価入力関数、および特徴量とラベルを使用してトレーニング演算がどのように定義されるかを `Estimator` に伝えるモデル関数です。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

`Estimator` をインスタンス化し、モデルをトレーニングします。

In [ ]:
estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

評価セットを使用してプログラムを評価します。

In [ ]:
estimator.evaluate(_eval_input_fn)

## TensorFlow 2: 組み込みの Keras メソッドを使用してトレーニングと評価を行う

この例では、TensorFlow 2 で Keras `Model.fit` と `Model.evaluate` を使用してトレーニングと評価を実行する方法を示します（詳細については、[組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)ガイドを参照してください）。

- `tf.data.Dataset` API を使用してデータセットパイプラインを準備することから始めます。
- 1 つの線形（<code>tf.keras.layers.Dense</code>）レイヤーを持つ単純な Keras <a>Sequential</a> モデルを定義します。
- Adagrad オプティマイザをインスタンス化します（`tf.keras.optimizers.Adagrad`）。
- `optimizer` 変数と平均二乗誤差（`"mse"`）損失を `Model.compile` に渡して、トレーニング用のモデルを構成します。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

これで、`Model.fit` を呼び出してモデルをトレーニングする準備が整いました。

In [ ]:
model.fit(dataset)

最後に、`Model.evaluate` を使用してモデルを評価します。

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## TensorFlow 2: カスタムトレーニングステップと組み込みの Keras メソッドを使用してトレーニングと評価を行う

TensorFlow 2 では、`tf.keras.callbacks.Callback` や `tf.distribute.Strategy` などの組み込みのトレーニングサポートを引き続き利用しながら、`tf.GradientTape` を使用して独自のカスタムトレーニングステップ関数を作成して、フォワードパスとバックワードパスを実行することもできます。（詳細については、[Model.fit の処理をカスタマイズする](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)および[トレーニングループの新規作成](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)を参照してください。)

この例では、`tf.keras.Sequential` をオーバーライドする `Model.train_step` をサブクラス化することにより、カスタム `tf.keras.Model` を作成することから始めます。（[tf.keras.Model のサブクラス化](https://www.tensorflow.org/guide/keras/custom_layers_and_models)について詳しくご覧ください)。そのクラス内で、データのバッチごとに 1 つのトレーニングステップでフォワードパスとバックワードパスを実行するカスタムの `train_step` 関数を定義します。


In [ ]:
class CustomModel(tf.keras.Sequential):
  """A custom sequential model that overrides `Model.train_step`."""

  def train_step(self, data):
    batch_data, labels = data

    with tf.GradientTape() as tape:
      predictions = self(batch_data, training=True)
      # Compute the loss value (the loss function is configured
      # in `Model.compile`).
      loss = self.compiled_loss(labels, predictions)

    # Compute the gradients of the parameters with respect to the loss.
    gradients = tape.gradient(loss, self.trainable_variables)
    # Perform gradient descent by updating the weights/parameters.
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # Update the metrics (includes the metric that tracks the loss).
    self.compiled_metrics.update_state(labels, predictions)
    # Return a dict mapping metric names to the current values.
    return {m.name: m.result() for m in self.metrics}

次に、前と同じように以下を実行します。

- `tf.data.Dataset` でデータセットパイプラインを準備します。
- 1 つの `tf.keras.layers.Dense` レイヤーで単純なモデルを定義します。
- Adagrad のインスタンス化（`tf.keras.optimizers.Adagrad`）
- 損失関数として平均二乗誤差（`"mse"`）を使用しながら、`Model.compile` でトレーニング用のモデルを構成します。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = CustomModel([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

`Model.fit` を呼び出してモデルをトレーニングします。

In [ ]:
model.fit(dataset)

最後に、`Model.evaluate` を使用してプログラムを評価します。

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## Next steps

役に立つと思われる追加の Keras リソースは次のとおりです。

- ガイド: [組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)
- ガイド: [Model.fit の処理をカスタマイズする](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- ガイド: [トレーニングループの新規作成](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
- ガイド: [サブクラス化によるレイヤーとモデルの新規作成](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

次のガイドは、`tf.estimator` API から分散ストラテジーのワークフローを移行するのに役立ちます。

- [TPUEstimator から TPUStrategy への移行](tpu_estimator.ipynb)
- [シングルワーカーのマルチ GPUトレーニングを移行する](mirrored_strategy.ipynb)
- [マルチワーカー CPU/GPUトレーニングを移行する](multi_worker_cpu_gpu_training.ipynb)